<center><a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a></center>

# 7. Assessment

Congratulations on going through today's course! Hopefully, you've learned some valuable skills along the way and had fun doing it. Now it's time to put those skills to the test. In this assessment, you will train a new model that is able to recognize fresh and rotten fruit. You will need to get the model to a validation accuracy of `92%` in order to pass the assessment, though we challenge you to do even better if you can. You will have the use the skills that you learned in the previous exercises. Specifically, we suggest using some combination of transfer learning, data augmentation, and fine tuning. Once you have trained the model to be at least 92% accurate on the validation dataset, save your model, and then assess its accuracy. Let's get started! 

In [1]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.v2 as transforms
import torchvision.io as tv_io

import glob
from PIL import Image

import utils

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

False

## 7.1 The Dataset

In this exercise, you will train a model to recognize fresh and rotten fruits. The dataset comes from [Kaggle](https://www.kaggle.com/sriramr/fruits-fresh-and-rotten-for-classification), a great place to go if you're interested in starting a project after this class. The dataset structure is in the `data/fruits` folder. There are 6 categories of fruits: fresh apples, fresh oranges, fresh bananas, rotten apples, rotten oranges, and rotten bananas. This will mean that your model will require an output layer of 6 neurons to do the categorization successfully. You'll also need to compile the model with `categorical_crossentropy`, as we have more than two categories.

<img src="./images/fruits.png" style="width: 600px;">

## 7.2 Load ImageNet Base Model

We encourage you to start with a model pretrained on ImageNet. Load the model with the correct weights. Because these pictures are in color, there will be three channels for red, green, and blue. We've filled in the input shape for you. If you need a reference for setting up the pretrained model, please take a look at [notebook 05b](05b_presidential_doggy_door.ipynb) where we implemented transfer learning.

In [2]:
from torchvision.models import vgg16
from torchvision.models import VGG16_Weights

weights = VGG16_Weights.DEFAULT
vgg_model = vgg16(weights=weights)

## 7.3 Freeze Base Model

Next, we suggest freezing the base model, as done in [notebook 05b](05b_presidential_doggy_door.ipynb). This is done so that all the learning from the ImageNet dataset does not get destroyed in the initial training.

In [3]:
# Freeze base model
vgg_model.requires_grad_(False)
next(iter(vgg_model.parameters())).requires_grad

False

## 7.4 Add Layers to Model

Now it's time to add layers to the pretrained model. [Notebook 05b](05b_presidential_doggy_door.ipynb) can be used as a guide. Pay close attention to the last dense layer and make sure it has the correct number of neurons to classify the different types of fruit.

The later layers of a model become more specific to the data the model trained on. Since we want the more general learnings from VGG, we can select parts of it, like so:

In [4]:
vgg_model.classifier[0:3]

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
)

Once we've taken what we've wanted from VGG16, then we can add our own modifications. No matter what additional modules we add, we still need to end with one value for each output.

In [5]:
N_CLASSES = 6

my_model = nn.Sequential(
    vgg_model.features,
    vgg_model.avgpool,
    nn.Flatten(),
    vgg_model.classifier[0:3],
    nn.Linear(4096, 500),
    nn.ReLU(),
    nn.Linear(500, N_CLASSES)
)
my_model

Sequential(
  (0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

## 7.5 Compile Model

Now it's time to compile the model with loss and metrics options. We have 6 classes, so which loss function should we use?

In [6]:
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(my_model.parameters())
my_model = torch.compile(my_model.to(device))

## 7.6 Data Transforms

To preprocess our input images, we will use the transforms included with the VGG16 weights.

In [7]:
pre_trans = weights.transforms()

Try to randomly augment the data to improve the dataset. Feel free to look at [notebook 04a](04a_asl_augmentation.ipynb) and [notebook 05b](05b_presidential_doggy_door.ipynb) for augmentation examples. There is also documentation for the [TorchVision Transforms class](https://pytorch.org/vision/stable/transforms.html).

**Hint**: Remember not to make the data augmentation too extreme.

In [8]:
IMG_WIDTH, IMG_HEIGHT = (224, 224)

random_trans = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomResizedCrop((IMG_HEIGHT, IMG_WIDTH), scale=(0.8, 1.0)),
])

## 7.7 Load Dataset

Now it's time to load the train and validation datasets. 

In [9]:
DATA_LABELS = ["freshapples", "freshbanana", "freshoranges", "rottenapples", "rottenbanana", "rottenoranges"] 
    
class MyDataset(Dataset):
    def __init__(self, data_dir):
        self.imgs = []
        self.labels = []
        
        for l_idx, label in enumerate(DATA_LABELS):
            data_paths = glob.glob(data_dir + label + '/*.png', recursive=True)
            for path in data_paths:
                img = tv_io.read_image(path, tv_io.ImageReadMode.RGB)
                self.imgs.append(pre_trans(img).to(device))
                self.labels.append(torch.tensor(l_idx).to(device))


    def __getitem__(self, idx):
        img = self.imgs[idx]
        label = self.labels[idx]
        return img, label

    def __len__(self):
        return len(self.imgs)

Select the batch size `n` and set `shuffle` either to `True` or `False` depending on if we are `train`ing or `valid`ating. For a reference, check out [notebook 05b](05b_presidential_doggy_door.ipynb).

In [10]:
n = 32

train_path = "data/fruits/train/"
train_data = MyDataset(train_path)
train_loader = DataLoader(train_data, batch_size=n, shuffle=True)
train_N = len(train_loader.dataset)

valid_path = "data/fruits/valid/"
valid_data = MyDataset(valid_path)
valid_loader = DataLoader(valid_data, batch_size=n, shuffle=False)
valid_N = len(valid_loader.dataset)

## 7.8 Train the Model

Time to train the model! We've moved the `train` and `validate` functions to our [utils.py](./utils.py) file. Before running the below, make sure all your variables are correctly defined.

It may help to rerun this cell or change the number of `epochs`.

In [11]:
epochs = 10
torch._dynamo.config.suppress_errors = True
for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    utils.train(my_model, train_loader, train_N, random_trans, optimizer, loss_function)
    utils.validate(my_model, valid_loader, valid_N, loss_function)

Epoch: 0


W0809 11:54:39.840000 5844 .venv\Lib\site-packages\torch\_dynamo\convert_frame.py:1280] WON'T CONVERT inner D:\PycharmProjects\NVIDIA-Fundamentals-of-Deep-Learning\.venv\Lib\site-packages\torch\_dynamo\external_utils.py line 68 
W0809 11:54:39.840000 5844 .venv\Lib\site-packages\torch\_dynamo\convert_frame.py:1280] due to: 
W0809 11:54:39.840000 5844 .venv\Lib\site-packages\torch\_dynamo\convert_frame.py:1280] Traceback (most recent call last):
W0809 11:54:39.840000 5844 .venv\Lib\site-packages\torch\_dynamo\convert_frame.py:1280]   File "D:\PycharmProjects\NVIDIA-Fundamentals-of-Deep-Learning\.venv\Lib\site-packages\torch\_dynamo\convert_frame.py", line 1213, in __call__
W0809 11:54:39.840000 5844 .venv\Lib\site-packages\torch\_dynamo\convert_frame.py:1280]     result = self._inner_convert(
W0809 11:54:39.840000 5844 .venv\Lib\site-packages\torch\_dynamo\convert_frame.py:1280]              ^^^^^^^^^^^^^^^^^^^^
W0809 11:54:39.840000 5844 .venv\Lib\site-packages\torch\_dynamo\convert_fr

Train - Loss: 22.5321 Accuracy: 0.7724
Valid - Loss: 12.5963 Accuracy: 0.7143
Epoch: 1
Train - Loss: 13.4207 Accuracy: 0.8596
Valid - Loss: 5.5847 Accuracy: 0.8815
Epoch: 2
Train - Loss: 8.9247 Accuracy: 0.9086
Valid - Loss: 3.4911 Accuracy: 0.8936
Epoch: 3
Train - Loss: 8.8269 Accuracy: 0.9095
Valid - Loss: 2.4725 Accuracy: 0.9210
Epoch: 4
Train - Loss: 5.8610 Accuracy: 0.9365
Valid - Loss: 1.9060 Accuracy: 0.9240
Epoch: 5
Train - Loss: 7.2149 Accuracy: 0.9289
Valid - Loss: 4.2733 Accuracy: 0.8936
Epoch: 6
Train - Loss: 6.4390 Accuracy: 0.9416
Valid - Loss: 2.6522 Accuracy: 0.9210
Epoch: 7
Train - Loss: 6.8802 Accuracy: 0.9349
Valid - Loss: 2.0868 Accuracy: 0.9240
Epoch: 8
Train - Loss: 6.4756 Accuracy: 0.9391
Valid - Loss: 3.2536 Accuracy: 0.9088
Epoch: 9
Train - Loss: 5.7854 Accuracy: 0.9450
Valid - Loss: 3.2643 Accuracy: 0.9149


## 7.9 Unfreeze Model for Fine Tuning

If you have reached 92% validation accuracy already, this next step is optional. If not, we suggest fine tuning the model with a very low learning rate.

In [12]:
# Unfreeze the base model
vgg_model.requires_grad_(False)
optimizer = Adam(my_model.parameters(), lr=.0001)

In [13]:
epochs = 1

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    utils.train(my_model, train_loader, train_N, random_trans, optimizer, loss_function)
    utils.validate(my_model, valid_loader, valid_N, loss_function)

Epoch: 0
Train - Loss: 4.3118 Accuracy: 0.9552
Valid - Loss: 3.4325 Accuracy: 0.9149


## 7.10 Evaluate the Model

Hopefully, you now have a model that has a validation accuracy of 92% or higher. If not, you may want to go back and either run more epochs of training, or adjust your data augmentation. 

Once you are satisfied with the validation accuracy, evaluate the model by executing the following cell. The evaluate function will return a tuple, where the first value is your loss, and the second value is your accuracy. To pass, the model will need have an accuracy value of `92% or higher`. 

In [ ]:
utils.validate(my_model, valid_loader, valid_N, loss_function)

## 7.11 Run the Assessment

To assess your model run the following two cells.

**NOTE:** `run_assessment` assumes your model is named `my_model`. If for any reason you have modified these variable names, please update the names of the arguments passed to `run_assessment`.

In [14]:
from run_assessment import run_assessment

ModuleNotFoundError: No module named 'run_assessment'

In [ ]:
run_assessment(my_model)

## 7.12 Generate a Certificate

If you passed the assessment, please return to the course page (shown below) and click the "ASSESS TASK" button, which will generate your certificate for the course.

<img src="./images/assess_task.png" style="width: 800px;">

<center><a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a></center>